In [ ]:
# 1.  Import files
from google.colab import files
uploaded = files.upload()
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['mining_block_model.csv']))
df.head()


Saving mining_block_model.csv to mining_block_model.csv


,Block_ID,X,Y,Z,Rock_Type,Ore_Grade (%),Tonnage,Ore_Value (¥/tonne),Mining_Cost (¥),Processing_Cost (¥),Waste_Flag,Profit (¥),Target
0,B00001,102,186,6,Magnetite,51.93,2131,294.48,53,38,0,433615.88,1
1,B00002,435,448,82,Hematite,59.05,1550,273.00,36,33,0,316200.00,1
2,B00003,348,476,94,Magnetite,63.79,2414,338.36,57,28,0,611611.04,1
3,B00004,270,127,98,Hematite,64.98,1297,307.60,30,29,0,322434.20,1
4,B00005,106,111,92,Waste,0.00,1309,0.00,67,28,1,-124355.00,0


Loaded the original mining block model CSV into a pandas DataFram using read_csv to start the cleaning and transformation process.

In [ ]:
# 2. Check basic structure
df.info()
df.describe()  # focuses on numeric columns (profit, tonnage, ore_grade, etc.)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Block_ID             75000 non-null  object 
 1   X                    75000 non-null  int64  
 2   Y                    75000 non-null  int64  
 3   Z                    75000 non-null  int64  
 4   Rock_Type            75000 non-null  object 
 5   Ore_Grade (%)        75000 non-null  float64
 6   Tonnage              75000 non-null  int64  
 7   Ore_Value (¥/tonne)  75000 non-null  float64
 8   Mining_Cost (¥)      75000 non-null  int64  
 9   Processing_Cost (¥)  75000 non-null  int64  
 10  Waste_Flag           75000 non-null  int64  
 11  Profit (¥)           75000 non-null  float64
 12  Target               75000 non-null  int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 7.4+ MB


,X,Y,Z,Ore_Grade (%),Tonnage,Ore_Value (¥/tonne),Mining_Cost (¥),Processing_Cost (¥),Waste_Flag,Profit (¥),Target
count,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000
mean,249.242680,248.773240,49.525360,45.992171,2000.475080,239.848683,49.563547,29.512120,0.200293,322005.651984,0.799707
std,144.613539,143.852756,28.858031,23.343018,577.028563,122.777574,11.526363,5.772658,0.400223,273713.865800,0.400223
min,0.000000,0.000000,0.000000,0.000000,1000.000000,0.000000,30.000000,20.000000,0.000000,-317152.000000,0.000000
25%,123.000000,124.000000,24.000000,50.920000,1501.000000,256.170000,40.000000,24.000000,0.000000,238527.937500,1.000000
50%,249.000000,248.000000,50.000000,55.620000,2003.000000,287.290000,50.000000,29.000000,0.000000,378189.050000,1.000000
75%,375.000000,373.000000,75.000000,60.340000,2501.000000,318.610000,60.000000,35.000000,0.000000,516542.122500,1.000000
max,499.000000,499.000000,99.000000,65.000000,2999.000000,350.000000,69.000000,39.000000,1.000000,883522.030000,1.000000


Inspected the dataset structure with info() and describe() to see column types, row count, and basic statistics (min, max, mean) for key numeric fields such as profit, tonnage, and ore grade

The dataset contains 75,000 rows and 13 columns with no missing values. Numeric ranges for ore grade, tonnage, costs, and profit appear realistic for a simulated mining block model, so I kept all rows for analysis.

In [ ]:
# 3. Check missing values per column
df.isna().sum()

,0
Block_ID,0
X,0
Y,0
Z,0
Rock_Type,0
Ore_Grade (%),0
Tonnage,0
Ore_Value (¥/tonne),0
Mining_Cost (¥),0
Processing_Cost (¥),0


Used isna().sum() to count missing values in each column so I can see if any important fields (like profit or tonnage) need special handling before analysis.

Checked for missing values in all columns using isna().sum() and found zero nulls in the dataset. This means no rows require imputation or removal due to missing data, so I can proceed directly to outlier checks and feature creation.

In [ ]:
# 4. Simple outlier check for profit using IQR

col = "Profit (¥)"

Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = df[(df[col] < lower) | (df[col] > upper)]
print("Number of profit outliers:", len(outliers))
outliers[[col]].head()


75000
Number of profit outliers: 5191


,Profit (¥)
15,-209755.0
35,-231088.0
36,-198616.0
37,-184920.0
39,-243389.0


Applied the standard IQR method on the profit column to detect potential outliers, defining outliers as values outside 1.5 × IQR from the first and third quartiles


In [1]:
import numpy as np

numeric_cols = ["Ore_Grade (%)", "Tonnage", "Ore_Value (¥/tonne)",
                "Mining_Cost (¥)", "Processing_Cost (¥)", "Profit (¥)"]

rows= []

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    mask = (df[col] < lower) | (df[col] > upper)
    rows.append({
        "column": col,
        "outlier_count": int(mask.sum()),
        "lower_bound": lower,
        "upper_bound": upper
    })

outlier_summary = pd.DataFrame(rows)
outlier_summary


NameError: name 'df' is not defined

Summarized outliers across key numeric columns using the IQR method. For each variable, I calculated a lower and upper bound (Q1 − 1.5 × IQR and Q3 + 1.5 × IQR) and counted how many values fall outside this range.

The results show that Ore_Grade (%) and Ore_Value (¥/tonne) each have 15,022 values outside their IQR bounds (grades above about 36.8% or ore values above about 162.5 ¥/tonne), indicating a long upper tail for high‑grade and high‑value blocks. In contrast, Tonnage, Mining_Cost (¥), and Processing_Cost (¥) have zero outliers within their respective bounds, suggesting these operational parameters are tightly controlled in the dataset. The Profit (¥) column has 5,191 outliers, with values below roughly −178k or above 933k, reflecting that some blocks are highly unprofitable or highly profitable but still plausible in a mining scenario.

In [ ]:
#show outliers for Profit (¥)
col = "Profit (¥)"

Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

profit_outliers = df[(df[col] < lower) | (df[col] > upper)]

profit_outliers[[col]].head(10)      # just profit
# or to see full context:
profit_outliers.head(10)


,Block_ID,X,Y,Z,Rock_Type,Ore_Grade (%),Tonnage,Ore_Value (¥/tonne),Mining_Cost (¥),Processing_Cost (¥),Waste_Flag,Profit (¥),Target
15,B00016,372,81,76,Waste,0.0,2305,0.0,65,26,1,-209755.0,0
35,B00036,252,421,72,Waste,0.0,2222,0.0,69,35,1,-231088.0,0
36,B00037,235,423,6,Waste,0.0,2257,0.0,59,29,1,-198616.0,0
37,B00038,344,410,37,Waste,0.0,2760,0.0,38,29,1,-184920.0,0
39,B00040,474,217,86,Waste,0.0,2363,0.0,64,39,1,-243389.0,0
43,B00044,187,377,39,Waste,0.0,2643,0.0,32,36,1,-179724.0,0
49,B00050,445,327,58,Waste,0.0,2457,0.0,44,38,1,-201474.0,0
55,B00056,130,169,33,Waste,0.0,2537,0.0,54,23,1,-195349.0,0
67,B00068,241,120,12,Waste,0.0,2484,0.0,61,38,1,-245916.0,0
70,B00071,52,493,70,Waste,0.0,2824,0.0,53,20,1,-206152.0,0


In [ ]:
#show outliers for Ore_grade
col = "Ore_Grade (%)"

Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

grade_outliers = df[(df[col] < lower) | (df[col] > upper)]

grade_outliers[[col]].head(10)      # just profit
# or to see full context:
grade_outliers.head(10)


,Block_ID,X,Y,Z,Rock_Type,Ore_Grade (%),Tonnage,Ore_Value (¥/tonne),Mining_Cost (¥),Processing_Cost (¥),Waste_Flag,Profit (¥),Target
4,B00005,106,111,92,Waste,0.0,1309,0.0,67,28,1,-124355.0,0
9,B00010,121,150,76,Waste,0.0,1865,0.0,58,31,1,-165985.0,0
15,B00016,372,81,76,Waste,0.0,2305,0.0,65,26,1,-209755.0,0
19,B00020,130,401,14,Waste,0.0,1661,0.0,39,35,1,-122914.0,0
26,B00027,293,195,88,Waste,0.0,2254,0.0,39,37,1,-171304.0,0
34,B00035,21,98,33,Waste,0.0,1564,0.0,41,25,1,-103224.0,0
35,B00036,252,421,72,Waste,0.0,2222,0.0,69,35,1,-231088.0,0
36,B00037,235,423,6,Waste,0.0,2257,0.0,59,29,1,-198616.0,0
37,B00038,344,410,37,Waste,0.0,2760,0.0,38,29,1,-184920.0,0
39,B00040,474,217,86,Waste,0.0,2363,0.0,64,39,1,-243389.0,0


In [ ]:
col = "Ore_Value (¥/tonne)"

Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

value_outliers = df[(df[col] < lower) | (df[col] > upper)]

value_outliers[[col]].head(10)
# or:
value_outliers.head(10)


,Block_ID,X,Y,Z,Rock_Type,Ore_Grade (%),Tonnage,Ore_Value (¥/tonne),Mining_Cost (¥),Processing_Cost (¥),Waste_Flag,Profit (¥),Target
4,B00005,106,111,92,Waste,0.0,1309,0.0,67,28,1,-124355.0,0
9,B00010,121,150,76,Waste,0.0,1865,0.0,58,31,1,-165985.0,0
15,B00016,372,81,76,Waste,0.0,2305,0.0,65,26,1,-209755.0,0
19,B00020,130,401,14,Waste,0.0,1661,0.0,39,35,1,-122914.0,0
26,B00027,293,195,88,Waste,0.0,2254,0.0,39,37,1,-171304.0,0
34,B00035,21,98,33,Waste,0.0,1564,0.0,41,25,1,-103224.0,0
35,B00036,252,421,72,Waste,0.0,2222,0.0,69,35,1,-231088.0,0
36,B00037,235,423,6,Waste,0.0,2257,0.0,59,29,1,-198616.0,0
37,B00038,344,410,37,Waste,0.0,2760,0.0,38,29,1,-184920.0,0
39,B00040,474,217,86,Waste,0.0,2363,0.0,64,39,1,-243389.0,0


In [ ]:
# 5. Remove extreme profit outliers (optional, if they look unrealistic)
col = "Profit (¥)"

Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df_clean = df[(df[col] >= lower) & (df[col] <= upper)].copy()
print("Rows before:", len(df), "Rows after:", len(df_clean))


Rows before: 75000 Rows after: 69809


Applied the IQR method to the Profit (¥) column and removed rows with extremely low or high profit values to reduce the influence of very rare profit cases on the analysis. However, I did not drop outliers in other numeric fields such as ore grade and ore value. In a mining context, very high grades or ore values often represent genuinely rich blocks rather than data errors, so I kept those rows to preserve important business information about high‑value opportunities.

In [ ]:
# 6. Map rock_type to business_category

mapping = {
    "Hematite": "Ore_Block",
    "Magnetite": "Ore_Block",
    "Waste": "Waste_Material"
}

df_clean["business_category"] = df_clean["Rock_Type"].map(mapping)
df_clean[["Rock_Type", "business_category"]].head()


,Rock_Type,business_category
0,Magnetite,Ore_Block
1,Hematite,Ore_Block
2,Magnetite,Ore_Block
3,Hematite,Ore_Block
4,Waste,Waste_Material


I added a new business_category column by mapping each rock_type to a business‑friendly label (for example, Hematite and Magnetite → Ore_Block, Waste → Waste_Material) using a dictionary and the pandas map function.

In [ ]:
# 7. Save cleaned dataset
from google.colab import files

df_clean.to_csv("mining_cleaned.csv", index=False)
files.download("mining_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved the cleaned and enriched dataset as mining_cleaned.csv without the index so it can be easily imported into PostgreSQL for further SQL analysis.